In [4]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from glob import glob
import subprocess

sns.set(style='whitegrid')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def find_csv(filename='consumo_hogar.csv'):
    """Devuelve una lista ordenada de rutas candidatas donde puede encontrarse el CSV."""
    candidates = [
        os.path.join('/mnt/data', filename),
        os.path.join('.', filename),
        os.path.join('/content', filename),
        os.path.expanduser(os.path.join('~', filename)),
    ]

    # Buscar recursivamente en cwd
    for root, dirs, files in os.walk('.'):
        if filename in files:
            candidates.append(os.path.join(root, filename))
            break

    # Buscar globalmente (puede demorarse en entornos grandes) — envuelto en try/except
    try:
        for path in glob(f'/**/{filename}', recursive=True):
            candidates.append(path)
    except Exception:
        pass

    # Filtrar duplicados manteniendo orden
    seen = set(); filtered = []
    for c in candidates:
        if c not in seen:
            seen.add(c); filtered.append(c)
    return filtered


In [ ]:
default_filename = 'consumo_hogar.csv'
csv_path = '/mnt/data/consumo_hogar.csv'  # valor por defecto; puedes cambiarlo antes de ejecutar

found_paths = find_csv(default_filename)
print('Rutas candidatas encontradas (en orden de comprobación):')
for p in found_paths:
    print('-', p)

loaded = False
for p in found_paths:
    if os.path.exists(p):
        try:
            df = pd.read_csv(p)
            csv_path = p
            loaded = True
            print(f"Archivo cargado correctamente desde: {p}")
            break
        except Exception as e:
            print(f"Se encontró {p} pero al leerlo ocurrió un error: {e}")

if not loaded:
    gen_script = 'generate_csv.py'
    if os.path.exists(gen_script):
        print(f"No se encontró {default_filename}; se intentará ejecutar {gen_script} para generarlo.")
        try:
            subprocess.run([sys.executable, gen_script], check=True)
            found_paths = find_csv(default_filename)
            for p in found_paths:
                if os.path.exists(p):
                    df = pd.read_csv(p)
                    csv_path = p
                    loaded = True
                    print(f"Archivo generado y cargado desde: {p}")
                    break
        except subprocess.CalledProcessError as e:
            print(f"Error ejecutando {gen_script}: {e}")
        except Exception as e:
            print(f"Error al intentar leer el CSV tras ejecutar {gen_script}: {e}")

if not loaded:
    print('
ATENCIÓN: No se encontró el archivo consumo_hogar.csv ni generate_csv.py.')
    print("Se generará un dataset sintético (consumo_hogar_generated.csv) para que el notebook funcione.")
    n = 60000
    rng = np.random.default_rng(42)
    temperatura = rng.normal(loc=20, scale=5, size=n).astype(float)
    # CORRECCIÓN: personas como float para permitir NaN
    personas = rng.integers(1, 6, size=n).astype(float)
    electrodomesticos = rng.integers(1, 15, size=n)

    consumo = (
        0.5 * temperatura +
        2.5 * personas +
        1.2 * electrodomesticos +
        0.02 * (electrodomesticos ** 2) +
        rng.normal(0, 3, size=n)
    )

    mask_nan = rng.choice(n, size=int(0.01 * n), replace=False)
    temperatura[mask_nan[: len(mask_nan)//2]] = np.nan
    personas[mask_nan[len(mask_nan)//2:]] = np.nan

    out_idx = rng.choice(n, size=50, replace=False)
    consumo[out_idx[:25]] *= 8
    consumo[out_idx[25:]] *= 0.1

    df = pd.DataFrame({
        'Temperatura': temperatura,
        'Personas': personas,
        'Electrodomesticos': electrodomesticos,
        'Consumo_kWh': consumo
    })
    generated_path = 'consumo_hogar_generated.csv'
    df.to_csv(generated_path, index=False)
    csv_path = os.path.abspath(generated_path)
    print(f"Dataset sintético generado y guardado en: {csv_path}")

print('
--- Vista rápida del dataframe cargado ---')
print(f'Usando archivo: {csv_path}')
print(df.head())
print('
Información básica:')
print(df.info())

In [ ]:
print('
--- Primeras 5 filas ---')
print(df.head())

print('
--- Info ---')
print(df.info())

print('
--- Describe (numéricas) ---')
print(df.describe())


In [ ]:
print('
--- Valores nulos por columna ---')
print(df.isnull().sum())

df_clean = df.copy()
expected_cols = ['Temperatura', 'Personas', 'Electrodomesticos', 'Consumo_kWh']
missing_expected = [c for c in expected_cols if c not in df_clean.columns]
if missing_expected:
    raise ValueError(f"Faltan columnas esperadas en el CSV: {missing_expected}. Revisa el archivo cargado.")

# Imputación planificada
impute_values = {}
impute_values['Temperatura'] = df_clean['Temperatura'].median() if df_clean['Temperatura'].isnull().sum() > 0 else None
impute_values['Personas'] = df_clean['Personas'].median() if df_clean['Personas'].isnull().sum() > 0 else None
impute_values['Electrodomesticos'] = df_clean['Electrodomesticos'].median() if df_clean['Electrodomesticos'].isnull().sum() > 0 else None
impute_values['Consumo_kWh'] = df_clean['Consumo_kWh'].median() if df_clean['Consumo_kWh'].isnull().sum() > 0 else None

print('
Imputación planificada (valores):')
print(impute_values)

for col, val in impute_values.items():
    if val is not None:
        df_clean[col].fillna(val, inplace=True)

print('
--- Después de imputación: nulos por columna ---')
print(df_clean[expected_cols].isnull().sum())


In [ ]:
plt.figure(figsize=(12,8))
for i, col in enumerate(expected_cols):
    plt.subplot(2,2,i+1)
    sns.boxplot(y=df_clean[col])
    plt.title(f'Boxplot - {col}')
plt.tight_layout()
plt.show()

from scipy import stats
if df_clean['Consumo_kWh'].nunique() > 1:
    z_scores = np.abs(stats.zscore(df_clean['Consumo_kWh']))
    outliers_z = np.where(z_scores > 3)[0]
    print(f"Cantidad de outliers por Z-score (>3) en 'Consumo_kWh': {len(outliers_z)}")
else:
    print("Consumo_kWh tiene 1 valor único; no se calcula Z-score.")

Q1 = df_clean['Consumo_kWh'].quantile(0.25)
Q3 = df_clean['Consumo_kWh'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers_iqr = df_clean[(df_clean['Consumo_kWh'] < lower_bound) | (df_clean['Consumo_kWh'] > upper_bound)]
print(f"Cantidad de outliers por IQR en 'Consumo_kWh': {outliers_iqr.shape[0]}")

lower_p = df_clean['Consumo_kWh'].quantile(0.01)
upper_p = df_clean['Consumo_kWh'].quantile(0.99)
print(f"Corte percentil 1%: {lower_p}, percentil 99%: {upper_p}")

df_cap = df_clean.copy()
df_cap['Consumo_kWh'] = df_cap['Consumo_kWh'].clip(lower=lower_p, upper=upper_p)

print('
--- Estadísticas antes y después del recorte (Consumo_kWh) ---')
print('Antes:
', df_clean['Consumo_kWh'].describe())
print('
Después:
', df_cap['Consumo_kWh'].describe())

In [ ]:
plt.figure(figsize=(14,4))
for i, feat in enumerate(['Temperatura', 'Personas', 'Electrodomesticos']):
    plt.subplot(1,3,i+1)
    sns.scatterplot(x=df_cap[feat], y=df_cap['Consumo_kWh'], alpha=0.3)
    try:
        sns.regplot(x=df_cap[feat], y=df_cap['Consumo_kWh'], scatter=False, lowess=True)
    except Exception:
        sns.regplot(x=df_cap[feat], y=df_cap['Consumo_kWh'], scatter=False)
    plt.title(f'{feat} vs Consumo_kWh')
plt.tight_layout()
plt.show()


In [ ]:
X = df_cap[['Temperatura', 'Personas', 'Electrodomesticos']].values
y = df_cap['Consumo_kWh'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
print('
--- Regresión Lineal Múltiple ---')
print('Coeficientes (Temperatura, Personas, Electrodomesticos):', lr.coef_)
print('Intercepto:', lr.intercept_)

y_pred_lr = lr.predict(X_test)


In [ ]:
poly2 = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('lin', LinearRegression())
])
poly2.fit(X_train, y_train)
y_pred_poly2 = poly2.predict(X_test)

poly3 = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler', StandardScaler()),
    ('lin', LinearRegression())
])
poly3.fit(X_train, y_train)
y_pred_poly3 = poly3.predict(X_test)

In [ ]:
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

mse_poly2 = mean_squared_error(y_test, y_pred_poly2)
r2_poly2 = r2_score(y_test, y_pred_poly2)

mse_poly3 = mean_squared_error(y_test, y_pred_poly3)
r2_poly3 = r2_score(y_test, y_pred_poly3)

results = pd.DataFrame({
    'Modelo': ['Regresión Lineal', 'Polinomio grado 2', 'Polinomio grado 3'],
    'MSE': [mse_lr, mse_poly2, mse_poly3],
    'R2': [r2_lr, r2_poly2, r2_poly3]
})
print('
--- Métricas de evaluación ---')
print(results)

In [ ]:
plt.figure(figsize=(15,4))
for i, (name, y_pred) in enumerate([('Lineal', y_pred_lr), ('Polinomio grado 2', y_pred_poly2), ('Polinomio grado 3', y_pred_poly3)]):
    plt.subplot(1,3,i+1)
    plt.scatter(y_test, y_pred, alpha=0.3)
    maxv = max(y_test.max(), y_pred.max())
    minv = min(y_test.min(), y_pred.min())
    plt.plot([minv, maxv], [minv, maxv], linestyle='--')
    plt.xlabel('Real')
    plt.ylabel('Predicho')
    plt.title(f'{name} (R2={[r2_lr, r2_poly2, r2_poly3][i]:.3f})')
plt.tight_layout()
plt.show()

In [ ]:
print('
Notas y recomendaciones:')
print('- Si la diferencia entre modelos polinómicos y lineal es pequeña, prefiera el modelo lineal por simplicidad.')
print('- Si observa overfitting (R2 muy alto en entrenamiento y bajo en prueba), reduzca grado o aplique regularización (Ridge/Lasso).')
print('- Ajuste la estrategia de tratamiento de outliers en función del contexto (si los outliers son reales, no deben eliminarse automáticamente).')

print('
Pregunta: ¿Deseas que el notebook obligue a usar tu propio archivo consumo_hogar.csv (fallando si no existe),')
print('o prefieres que intente generar un dataset sintético automáticamente cuando no encuentre el archivo?')
